In [13]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import json

In [5]:
page = requests.get("https://www.conseil-constitutionnel.fr/les-membres-depuis-1959")

In [6]:
membres = re.findall("(?:/membres/)([a-z\-]*)",str(page.content))

In [7]:
listeMembres2 = []

for membre in tqdm(membres):
    page = requests.get(f'https://www.conseil-constitutionnel.fr/membres/{membre}')
    soup = BeautifulSoup(page.content, 'html.parser')
    pers = {}

    nom = soup.find('h1', {'class':'title'})
    bio = soup.find('div',{'class':re.compile('naissance')})
    typeMembre = soup.find('div',{'class':re.compile('type-membre')})
    fonction = soup.find('div',{'class':re.compile('fonction')})
    nomination = soup.find('div',{'class':re.compile('nomination')})
    serment = soup.find('div',{'class':re.compile('serment')})
    for el,val in {'nom':nom,'bio':bio,'typeMembre':typeMembre,'fonctionDebut':fonction,'nomination':nomination,'serment':serment}.items():
        if val:
            pers[el]=val.getText()

    listeMembres2.append(pers)

#listeMembres2

100%|██████████| 89/89 [00:31<00:00,  2.80it/s]


In [17]:
# json.dumps(listeMembres2, ensure_ascii=False)
with open('listeMembres2.json', 'w', encoding='utf8') as json_file:
    json.dump(listeMembres2, json_file, ensure_ascii=False)

In [14]:
with open('listeMembres2.json', encoding='utf-8') as fh:
    listeMembres2 = json.load(fh)

In [15]:
from datetime import datetime
import locale
locale.setlocale(locale.LC_TIME,'fr_FR')

'fr_FR'

In [16]:
def transfoDate(date:str):
    try:
        dateRe = str(datetime.date(datetime.strptime(re.sub('(1)er',r'\1',date), "%d %B %Y")))
        return dateRe
    except:
        print(date)
        return date

In [ ]:
listeMembres3 = []
regex = '[0-9]{0,2}[er]* [A-Za-zéèêàû]+ [0-9]{4}'

for membre2 in tqdm(listeMembres2):
    membre3={}

    nomMO = re.search('([A-Z]{2,}.*)',membre2['nom'])
    membre3['prenom']=membre2['nom'][:nomMO.start()-1]
    membre3['nom']=membre2['nom'][nomMO.start():]

    dates = re.findall(regex, membre2['bio'])
    if len(dates)>1:
        membre3['dateDeces'] = transfoDate(dates[-1])
    membre3['dateNaissance'] = transfoDate(dates[0])
    membre3['lieuNaissance'] = re.split('à |au |, ',membre2['bio'])[1].strip()

    for el in [le for le in ['fonctionDebut','typeMembre','serment','nomination'] if le in membre2.keys()]:
        contenu = re.findall(regex, membre2[el])
        if contenu:
            if len(contenu)>1:
                membre3[f'{el}Debut']=transfoDate(contenu[0])
                membre3[f'{el}Fin']=transfoDate(contenu[-1])
            else:
                membre3[el]=transfoDate(contenu[0])
        else:
            membre3[el]=membre2[el]

    if all(x in ['serment'] for x in membre2.keys()):
        membre3['nominationPar'] = re.split('par ',membre2['nomination'])[1]

    listeMembres3.append(membre3)

#listeMembres3

In [45]:
with open('listeMembres3.json', 'w', encoding='utf8') as json_file:
    json.dump(listeMembres3, json_file, ensure_ascii=False)

In [2]:
import json
import re
from slugify import slugify

In [4]:
with open('listeMembres3.json', encoding='utf-8') as fh:
    listeMembres3 = json.load(fh)

In [10]:
indent = 0
tab = '\t'

with open('listeMembres3.xml', 'w', encoding='utf8') as file:
    file.write(f'{tab*indent}<listPerson>\r')
    
indent += 1
for membre in listeMembres3:
    if 'typeMembre' not in membre.keys():
        membre['typeMembre']='membre'
    else:
        membre['typeMembre']=slugify(membre['typeMembre'])

    s=f'{tab*indent}<person xml:id="{slugify(membre["nom"])}_{slugify(membre["prenom"])}">\r'
    indent += 1
    s+=f'{tab*indent}<persName>\r'
    indent += 1
    s+=f'{tab*indent}<surname>{membre["nom"].capitalize()}</surname>\r'
    s+=f'{tab*indent}<forename>{membre["prenom"]}</forename>\r'
    indent -= 1
    s+=f'{tab*indent}</persName>\r'
    s+=f'{tab*indent}<sex value="{membre["sexe"]}"/>\r'

    s+=f'{tab*indent}<birth when="{membre["dateNaissance"]}">\r'
    indent += 1
    s+=f'{tab*indent}<placeName>{membre["lieuNaissance"]}</placeName>\r'
    indent -= 1
    s+=f'{tab*indent}</birth>\r'

    if 'dateDeces' in membre.keys():
        s+=f'{tab*indent}<death when="{membre["dateDeces"]}"></death>\r'

    st= f'<affiliation role="{membre["typeMembre"].lower()}" ref="?" from="{membre["fonctionDebut"]}"'
    if 'fonctionFin' in membre.keys():
        st+= f' to="{membre["fonctionFin"]}"'
    s+=f'{tab*indent}{st}/>\r'
    indent -= 1
    s+=f'{tab*indent}</person>\r'

    with open('listeMembres3.xml', 'a', encoding='utf8') as file:
        file.write(s)
indent-=1

with open('listeMembres3.xml', 'a', encoding='utf8') as file:
    file.write(f'{tab*indent}</listPerson>')